<a href="https://colab.research.google.com/github/Moimschoh/data-analyst/blob/main/pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and paths

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import glob
from IPython.display import display
import os
import sys
from datetime import date,timedelta
import json
import numpy as np
import time
from math import cos, sin
from os import path


In [ ]:
#Path Definition

EEDA_PATH = r"C:\Users\49178\Downloads\Telegram Desktop\LoopARData"
ET_DATA_PATH = EEDA_PATH + r'\EyeTracking\eye'
SCENE_EVENT_DATA_PATH = EEDA_PATH +  r'\SceneData\scene\HUDOnly'
STEERING_DATA_PATH = EEDA_PATH + r'\Input\input'
PARTICIPANT_DATA_PATH = EEDA_PATH + r'\ParticipantCalibrationData\cakibration'
PROCESSED_DATA_PATH = '..\csv_et'
COMBINED_DATA_PATH = '..\et_comb'
EVENT_DURATION_DATA_PATH = '..\Event_dur'
ONLY_EVENTS_DATA_PATH = r'C:\Users\49178\Downloads\Telegram Desktop\LoopARData\FullLoopAR1\HUDOnly'
Cut_EVENTS_DATA_PATH = r'C:\Users\49178\Downloads\Telegram Desktop\LoopARData\FullLoopAR1\Event_Cutted\HUDOnly'

EVENTS_OBJECT_PATH = '..\Evt_Objects'

#Man_Path = r'E:\EEDA_Data\Manual_Scenes'
#Auto_Path = r'E:\EEDA_Data\Autonomous_scenes'

PLOT_PATH = '..\PLOTS\data_quality'
os.makedirs(os.path.dirname(PROCESSED_DATA_PATH),exist_ok=True)
os.makedirs(os.path.dirname(COMBINED_DATA_PATH),exist_ok=True)
os.makedirs(os.path.dirname(PLOT_PATH), exist_ok=True)

In [ ]:
ONLY_EVENTS_DATA_PATH

'C:\\Users\\49178\\Downloads\\Telegram Desktop\\LoopARData\\FullLoopAR1\\HUDOnly'

In [ ]:
import os
os.getcwd()

'C:\\Users\\49178\\Desktop\\mastertheses'

# Functions

## checkEyeError & checkIncomplete

In [ ]:
# here we apply our eye validation criteria and we drop unrelated columns
def checkEyeError(files):
    df = pd.DataFrame()
    for fi in files: # reading all files
        with open(fi, 'r') as j:
            tmp = json.loads(j.read())

            tempdf0 = pd.DataFrame.from_dict(tmp,orient='index').T

            tempdf0[["EyeValidationError.x","EyeValidationError.y","EyeValidationError.z"]] = pd.DataFrame(tempdf0["EyeValidationError"].to_list(), columns=['x', 'y','z'])

            if(tempdf0["EyeValidationError.x"][0]<1.5) and (tempdf0["EyeValidationError.y"][0]<1.5):
                df = pd.concat([df, tempdf0], ignore_index=True)


    df = df.drop(['IPAddress', 'VRmode',
           'SteeringInputDevice', 'EyeValidationError', 'SeatCalibrationOffset',
           'ExperimentDuration',
                  'ApplicationDuration',
           'AverageExperimentFPS',
                  'SpecialNotes', 'EyeValidationError.z'], axis=1)
    return df
    #display(df)

In [ ]:
# The function 'checkIncomplete' identifies incomplete data records in a DataFrame df based on the number of files associated with each participant.
# If a participant has more than 3 files, their index is stored, and a new DataFrame with only these participants is returned.
def checkIncomplete(path,df):
    des_index = []
    for i in range(0,df['ParticipantUuid'].count()):
        name = df['ParticipantUuid'][i]
        Liste = SelectAllFilesOfSameName(name,path)
        if len(Liste)>3:
            des_index.append(i)
    df = df.iloc[des_index]
    df = df.reset_index(drop=True)
    return(df)

In [ ]:
# The function incompletePersonUID reads all .txt files from a specified path, checks if the 'EventBehavior' list in the JSON content of each file has fewer than 3 items, and returns a list of unique identifiers for the files that meet this condition.
def incompletePersonUID(path):
    files = glob.glob(f'{path}\*.txt')

    incomplete_files = []

    for file in files:
        with open(file, 'r') as j:
            scene_data = json.loads(j.read())

            event_behaviour_list = scene_data['EventBehavior']

            if not len(event_behaviour_list) >= 3:
                incomplete_files.append(os.path.basename(file).split("_")[0])
    return incomplete_files

In [ ]:
# here we remove all the incomplete files that we found already
def removeIncompleteUIDsFromDF(incomplete, df):
    for uid in incomplete:
        df.drop(df[df['ParticipantUuid'] == uid].index, inplace = True)


## selectConditions & SelectAllFilesOfSameName & SelectAllCSVOfSameName

In [ ]:
# here we define our experimental conditions
def selectConditions(df, ConditionName):
    df = df[(df.ExperimentalCondition == ConditionName)]#df[(df.ExperimentalCondition == 'Autonomous')] |]
    df = df.reset_index(drop=True)
    return df

In [ ]:
# here we put all the files with the same name in dataframe
def SelectAllFilesOfSameName(name,path):
    files = glob.glob(f'{path}/*.txt')
    mylist = list()
    for fi in files[:]: # reading all files
        if name in fi:
            mylist.append(fi)
    return mylist

In [ ]:
# The function SelectAllCSVOfSameName reads all CSV files from a specified directory and returns a list of their paths.
def SelectAllCSVOfSameName(path):
    files = glob.glob(f'{path}/*.csv')
    mylist = list()
    for fi in files[:]: # reading all files
        mylist.append(fi)
    return mylist

In [ ]:
# here we drop some unrelated uncolumns
ToDrop = ['FPS',#'pupilDiameterMillimetersLeft', 'pupilDiameterMillimetersRight',
  'hitObjects']
ToDrop2 = [ 'UnixTimeStamp', 'TobiiTimeStamp',
       'EyeOpennessLeftSranipal', 'EyeOpennessRightSranipal', 'HmdPosition_x',
       'HmdPosition_y', 'HmdPosition_z', 'NoseVector_x', 'NoseVector_y',
       'NoseVector_z', 'EyePositionCombinedWorld_x',
       'EyePositionCombinedWorld_y', 'EyePositionCombinedWorld_z', 'EyePositionCombinedLocal_x',
       'EyePositionCombinedLocal_y', 'EyePositionCombinedLocal_z',
       'CarRotation_x', 'CarRotation_y', 'CarRotation_z',
       'CarRotation_w',
       'is_blink']

## LoadEyetrackingData & TobiiQuality

In [ ]:
# then the function 'LoadEyetrackingData' reads a list of JSON files, processes the eye-tracking data, and performs various transformations and interpolations on it.
# it also converts Unix timestamps to datetime.
# then it sorts the DataFrame by timestamp and resets the index.
# it calculates the time from the start for each UID and scene.
#  it adds the interpolated columns to the DataFrame.
# at the end, the code will load and process the eye-tracking data from the specified JSON files, performing the necessary transformations and interpolations.

In [ ]:
def LoadEyetrackingData(mylist):
    start = time.process_time()
    et_df = pd.DataFrame()
    #print(" Before Reading :",time.process_time() - start)
    for fi in mylist:
        with open(fi, 'r') as j:
            tmp = json.loads(j.read())
        for nS, sample in enumerate(tmp):
# nano second-based time format that specifies the precision of the DateTime object.
            tmpdf = pd.json_normalize(data=sample, sep='_')

            tmpdf = tmpdf.drop(ToDrop, axis=1)

            et_df = pd.concat([et_df, tmpdf], ignore_index=True)

    if not et_df.empty:
        et_df = et_df.drop_duplicates()
        et_df['uid'] = fi.split('\\')[-1].split('_')[0]
        et_df['timestamp'] = pd.to_datetime(et_df.UnixTimeStamp,
                                            utc=True,
                                            unit='s',
                                            origin='unix')
        et_df = et_df.sort_values(by=['timestamp'])
        et_df = et_df.reset_index(drop=True)
        et_df.set_index('timestamp', inplace=True)
        #print(" Before Timestamp :",time.process_time() - start)
        et_df['time_from_start'] = (et_df
        #                                    .reset_index()
        #                                   .groupby(['uid','scene',], as_index=False)
        #                                    .timestamp
                                   .transform(lambda x:
                                              (x.index - x.index[0])/np.timedelta64(1,'s'))
                                   .UnixTimeStamp
                                   )
        et_df = et_df.reset_index()
        et_df[[
            'EyeDirectionCombinedLocal_x_copy',
            'EyeDirectionCombinedLocal_y_copy',
            'EyeDirectionCombinedLocal_z_copy'
        ]] = et_df[[
              'EyeDirLocalCombined_x', 'EyeDirLocalCombined_y',
            'EyeDirLocalCombined_z'
        ]]

        #print(" Before Interpolation :",time.process_time() - start)
        tmpdf = (et_df.groupby([
            'uid'
        ])['EyeDirectionCombinedLocal_x_copy',
           'EyeDirectionCombinedLocal_y_copy',
           'EyeDirectionCombinedLocal_z_copy'].transform(
               lambda x: x.interpolate(method='polynomial',
                                       order=3,
                                       limit_direction='both')).reset_index())
        et_df[[
            'combinedEyeDir_x_intp', 'combinedEyeDir_y_intp',
            'combinedEyeDir_z_intp'
        ]] = tmpdf[[
            'EyeDirectionCombinedLocal_x_copy',
            'EyeDirectionCombinedLocal_y_copy',
            'EyeDirectionCombinedLocal_z_copy'
        ]]
        #et_df = et_df.drop(ToDrop2,axis=1)
        #print(" After Interpolation :",time.process_time() - start)
        return et_df
        #return et_df

In [ ]:
def TobiiQuality(et_df,df,name):
    o = et_df.query("TobiiTimeStamp==0")
    o = o.shape[0]
    o = o/len(et_df.index)*100
    df.loc[df['ParticipantUuid'] == name, ['TobiiQuality Bads in %']] = o

    return df

## ReadSteeringData & CombineSteeringAndET

In [ ]:
#  the function 'ReadSteeringData' reads JSON files containing steering data, processes them into a pandas DataFrame, and performs various transformations.

In [ ]:
def ReadSteeringData(mylist):
    start = time.process_time()
    et_df = pd.DataFrame()
    for fi in mylist:
        with open(fi, 'r') as j:
            tmp = json.loads(j.read())
        for nS, sample in enumerate(tmp):
            tmpdf = pd.json_normalize(data=sample, sep='_')
            et_df = pd.concat([et_df, tmpdf], ignore_index=True)


    if not et_df.empty:
        et_df = et_df.drop_duplicates()
        et_df['uid'] = fi.split('\\')[-1].split('_')[0]
        et_df['timestamp'] = pd.to_datetime(et_df.TimeStamp, utc=True, unit='s', origin='unix')
        et_df = et_df.sort_values(by=['timestamp'])
        et_df = et_df.reset_index(drop=True)
        et_df = et_df.reset_index(drop=True)

        #print(" Steering Reading :",time.process_time() - start)
        return et_df



In [ ]:
# The function 'CombineSteeringAndET'  aims to combine eye-tracking data (et_df) and steering data (st_df) into a single DataFrame based on their timestamps.

In [ ]:
def CombineSteeringAndET(et_df,st_df):


    st_df.set_index('timestamp', inplace=True)

    #Check if DF are equally long
    b = st_df.shape[0]- et_df.shape[0]
    st_df.drop(st_df.tail(b).index,inplace=True)

    st_df = st_df.reset_index()

    et_df['Equal'] = np.where(et_df['timestamp'] == st_df['timestamp'],True,False)
    et_df[['SteeringInput', 'AcellerationInput', 'BrakeInput']] = st_df[['SteeringInput', 'AcellerationInput', 'BrakeInput']]
    return et_df


In [ ]:
# The function 'CombinationQuality' aims to evaluate the compatibility between steering and eye-tracking timestamps and update a DataFrame with the results.

In [ ]:
def CombinationQuality(et_df,df,name):
    f = et_df.query('Equal==False')
    f = f.shape[0]
    f = 100 - f/len(et_df.index)*100
    df.loc[df['ParticipantUuid'] == name, ['Steering and Eytracking Timestamp compability in %']] = f
    return df

## ReadEventData

In [ ]:
# The function ReadEventData processes a list of JSON files to extract and compile event-related data into two main DataFrames.

In [ ]:
def ReadEventData(mylist):
    interims_df = pd.DataFrame()
    Event_dur_df = pd.DataFrame()
    Event_object_movement_df = pd.DataFrame()
    event_samples_df = pd.DataFrame()
    for fi in mylist:
        with open(fi, 'r') as j:
            tmp = json.loads(j.read())
            x = tmp.get('EventBehavior')
            for p in range(len(x)):
                tmpdf = pd.json_normalize(data=x[p], sep='_')
                interims_df = tmpdf[['EventName', 'StartofEventTimeStamp', 'EndOfEventTimeStamp',
           'EventDuration', 'SuccessfulCompletionState', 'HitObjectName']]
                Event_dur_df = pd.concat([Event_dur_df,interims_df],ignore_index = True)
                tmpdf2 = pd.json_normalize(tmpdf["dynamicObjectData"])
                for nS, sample in enumerate(tmpdf2):
                    temp = list(tmpdf2[nS])
                    tmpdf3 = pd.json_normalize(data=temp, sep='_')
                    tmpdf3['uid'] = fi.split('\\')[-1].split('_')[0]
                    tmpdf3['scene'] = fi.split('_')[-1].split('.')[0]
                    tmpdf3[['EventName', 'StartofEventTimeStamp', 'EndOfEventTimeStamp',
           'EventDuration', 'SuccessfulCompletionState', 'HitObjectName']] = interims_df[['EventName', 'StartofEventTimeStamp', 'EndOfEventTimeStamp',
           'EventDuration', 'SuccessfulCompletionState', 'HitObjectName']]
                    event_samples_df = pd.concat([event_samples_df, tmpdf3], ignore_index=True)
    if not event_samples_df.empty:
        event_samples_df = event_samples_df.drop_duplicates()

        event_samples_df['timestamp'] = pd.to_datetime(event_samples_df.UnixTimeStamp, utc=True, unit='s', origin='unix')
        event_samples_df['StartofEventTime'] = pd.to_datetime(event_samples_df.StartofEventTimeStamp, utc=True, unit='s', origin='unix')
        event_samples_df['EndofEventTime'] = pd.to_datetime(event_samples_df.EndOfEventTimeStamp, utc=True, unit='s', origin='unix')
        event_samples_df = event_samples_df.sort_values(by=['timestamp'])

        Event_object_movement_df = event_samples_df[['uid','EventName','timestamp','ObjectName0', 'ObjectName1', 'ObjectName2',
       'ObjectPosition0.x', 'ObjectPosition0.y', 'ObjectPosition0.z','ObjectPosition1.x', 'ObjectPosition1.y',
       'ObjectPosition1.z','ObjectPosition2.x', 'ObjectPosition2.y', 'ObjectPosition2.z']]

        event_samples_df = event_samples_df.drop(['UnixTimeStamp', 'ObjectName0', 'ObjectName1', 'ObjectName2',
       'ObjectPosition0.x', 'ObjectPosition0.y', 'ObjectPosition0.z','timestamp',
       'ObjectRotation0.x', 'ObjectRotation0.y', 'ObjectRotation0.z',
       'ObjectRotation0.w', 'ObjectPosition1.x', 'ObjectPosition1.y',
       'ObjectPosition1.z', 'ObjectRotation1.x', 'ObjectRotation1.y',
       'ObjectRotation1.z', 'ObjectRotation1.w', 'ObjectPosition2.x',
       'ObjectPosition2.y', 'ObjectPosition2.z', 'ObjectRotation2.x',
       'ObjectRotation2.y', 'ObjectRotation2.z', 'ObjectRotation2.w',  'StartofEventTimeStamp', 'EndOfEventTimeStamp',
        'HitObjectName'
       ],axis=1)


        #event_samples_df.to_csv(path)
        #event_samples_df = event_samples_df[(event_samples_df.EventName == 'Event_2')|(event_samples_df.EventName == 'Event_6')|(event_samples_df.EventName == 'Event_8')|(event_samples_df.EventName == 'Event_10')]
        Event_object_movement_df = Event_object_movement_df.drop_duplicates().reset_index(drop=True)
        event_samples_df = event_samples_df.drop_duplicates().reset_index(drop=True)
        #Event_object_movement_df.to_csv(path2)
        return(event_samples_df,Event_object_movement_df)

In [ ]:
# The function EyeThetaCalculation calculates horizontal and vertical angles (theta) for eye direction vectors and their interpolated values from an eye-tracking dataset.

In [ ]:
def EyeThetaCalculation(et_samples_df):
    et_samples_df.reset_index(inplace=False)
    et_samples_df['eye_theta_h'] = np.arctan2(et_samples_df['EyeDirectionCombinedLocal_x_copy'],
                                           et_samples_df['EyeDirectionCombinedLocal_z_copy'])
    et_samples_df['eye_theta_v'] = np.arctan2(et_samples_df['EyeDirectionCombinedLocal_y_copy'],
                                           et_samples_df['EyeDirectionCombinedLocal_z_copy'])
    et_samples_df['eye_theta_h_int'] = np.arctan2(et_samples_df['combinedEyeDir_x_intp'],
                                           et_samples_df['combinedEyeDir_z_intp'])
    et_samples_df['eye_theta_v_int'] = np.arctan2(et_samples_df['combinedEyeDir_y_intp'],
                                           et_samples_df['combinedEyeDir_z_intp'])
    return et_samples_df

In [ ]:
# here we’re working on a function to compute the eye theta angle in the world coordinate system from a DataFrame.
# The function 'EyeThetaCalculationWorld' calculates the angle using the arctangent function

In [ ]:
def EyeThetaCalculationWorld(et_samples_df):
    et_samples_df.reset_index(inplace=False)
    et_samples_df['eye_theta_h_World'] = np.arctan2(et_samples_df['EyeDirectionCombinedWorld_x'],
                                           et_samples_df['EyeDirectionCombinedWorld_z'])

    return et_samples_df

In [ ]:
# function ThetaQuality calculates the percentage of rows in et_df where the combinedEyeDir_x_intp value is greater than 1, and then updates a DataFrame df with this percentage for a specific participant identified by name.
def ThetaQuality(et_df,df,name):
    g = et_df.query("combinedEyeDir_x_intp>1")
    g = g.shape[0]
    g = g/len(et_df.index)*100
    df.loc[df['ParticipantUuid'] == name, ['Theta over 1 in %']] = g
    #print(name)
    return df


In [ ]:
#https://stackoverflow.com/questions/40375366/pandas-to-csv-checking-for-overwrite
#def write_csv_df(path,filename, O_df):
    # Give the filename you wish to save the file to
    #pathfile = os.path.normpath(os.path.join(path))

    # Use this function to search for any files which match your filename
    #files_present = os.path.isfile(pathfile)
    # if no matching files, write to csv, if there are matching files, print statement
   # if not files_present:
    #    O_df.to_csv(pathfile, sep=';')
   # else:
     #   overwrite = raw_input("WARNING: " + pathfile + " already exists! Do you want to overwrite <y/n>? \n ")
     #   if overwrite == 'y':
          #  O_df.to_csv(pathfile, sep=';')
       # elif overwrite == 'n':
         #   new_filename = input("Please enter a name:\n")
          #  write_csv_df(path,new_filename,O_df)
       # else:
        #    print ("Not a valid input. Data is NOT saved!\n")


# codes

In [ ]:
# The below code snippet outlines a series of data processing steps for participant files, involving filtering, checking errors, and selecting specific conditions.

In [ ]:
# Loads Participants Files
Participant_files = glob.glob(f'{PARTICIPANT_DATA_PATH}/*.txt')
#files = glob.glob(f'{ONLY_EVENTS_DATA_PATH}\*.csv')


incomplete_personUID = incompletePersonUID(SCENE_EVENT_DATA_PATH)
print(incomplete_personUID)

df = pd.DataFrame()
#print(df)
# Filters for EyeValidation error above 1.5
df = checkEyeError(Participant_files[:])
#print("After check eye error", df)
# checks if Participants made it thorugh till the end
df = checkIncomplete(ET_DATA_PATH,df)
#print("After check incomplete", df)
# Filter for the wanted condition
Conditionname ='HUDOnly'
df = selectConditions(df,Conditionname)

#display(df)

"""
if uid in incomplete_personUID -> delete row from df
"""

removeIncompleteUIDsFromDF(incomplete_personUID, df)



['5fe54d902792467181089d8e4a73b80c', '6f36a58eb7d843a593a01191c53bc0d0']
                    ParticipantUuid ExperimentalCondition  \
0  29006bddb3e8430582165c73bc7af864               HUDOnly   
1  4ed7e605b1614ccc98269519940f6965               HUDOnly   
4  7c4a213768e645e4a46f4317ad87d76d               HUDOnly   

  TrainingSuccessState NumberOfTrainingTrials  EyeValidationError.x  \
0                 True                      2              1.363390   
1                 True                      1              0.562467   
4                 True                      1              0.781845   

   EyeValidationError.y  
0              1.362076  
1              0.554869  
4              1.025252  


In [ ]:
df


,ParticipantUuid,ExperimentalCondition,TrainingSuccessState,NumberOfTrainingTrials,EyeValidationError.x,EyeValidationError.y
0,29006bddb3e8430582165c73bc7af864,HUDOnly,True,2,1.363390,1.362076
1,4ed7e605b1614ccc98269519940f6965,HUDOnly,True,1,0.562467,0.554869
4,7c4a213768e645e4a46f4317ad87d76d,HUDOnly,True,1,0.781845,1.025252


In [ ]:
# The code snippet processes data for multiple participants, iterating through various scenes and conditions, and saving the results to CSV files.

In [ ]:
#files_present = os.path.isfile(path)
df_A = pd.DataFrame()
# timekeeper for overall process
start2 = time.process_time()
# Goes through each participants

for index, row in df.iterrows():

    name = row[0]
    Conditionname = row[1]
    print(name)
    print(Conditionname)
    #timekeeper per iteration
    start = time.process_time()
    # Select desired Scenes
    scenes =  ['CountryRoad',
              'MountainRoad',
              'Westbrueck','Autobahn']


    neList = list()
    # Get File lists
    CompL = SelectAllFilesOfSameName(name,ET_DATA_PATH)
    CompLSteer = SelectAllFilesOfSameName(name,STEERING_DATA_PATH)
    CompLEvents = SelectAllFilesOfSameName(name,SCENE_EVENT_DATA_PATH)
    CompLEventCuts = SelectAllCSVOfSameName(ONLY_EVENTS_DATA_PATH)
    # Goes through each scene for each particpant
    for i in scenes:
        # get all needed Eyetracking files
        for b in CompL:
            if i in b:
                neList = list()
                neList.append(b)
                et_df = LoadEyetrackingData(neList)
                #print("Loaded Eyetracking")
        # get all needed steering files
        for c in CompLSteer:
            if i in c:
                neList = list()
                neList.append(c)
                st_df = ReadSteeringData(neList)
        # Possible error in some files when combining steering and eyetracking data
        try:
            et_df = CombineSteeringAndET(et_df,st_df)
        except:
            print("error")
            continue
        # get all needed Event files
 ##       for d in CompLEvents:
 ##           if i in d:
 ##               neList = list()
 ##               neList.append(d)
 ##               evt_df,mvmt_df = ReadEventData(neList)

        # Calculate the Theta values
        et_df = EyeThetaCalculation(et_df)
        # Clears Theta values greater then 1 and smaller then -1
        ## et_df = ThetaWorldToNan(et_df,'eye_theta_h')

        # old function for the Event snippets
        # evt_df = CalculateEventTime(et_df,evt_df)
        # Claculates 'Distance' values = Euklidian distance summ of x and z position of the car
       # et_df = Distance(et_df)

        # check if a scene-file is already existing, if not create it
        path = os.path.join(ONLY_EVENTS_DATA_PATH,Conditionname+'_'+ name + i+'.csv')
        if "..\\FullLoopAR1"+i+".csv" in CompLEventCuts:
            print(path)

            O_df = pd.read_csv(path)
            print("Found file ", i)
        else :
            O_df = pd.DataFrame()
            print("New File")#
        # add new participant to others
        O_df = pd.concat([O_df, et_df], ignore_index=True)
        # Save the file to ONLY_EVENTS_DATA_PATH
        O_df['Condition'] = Conditionname
        O_df['Scene'] = i
        O_df.to_csv(path, index = False)
       # O_df = write_csv_df (path,filename,O_df)

    print("One Iteration needs: ",time.process_time() - start)

    print(index)
print("Alles in: ",time.process_time() - start2)


29006bddb3e8430582165c73bc7af864
HUDOnly


C:\Users\49178\AppData\Local\Temp\ipykernel_69120\1783519745.py:46: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  tmpdf = (et_df.groupby([


New File


C:\Users\49178\AppData\Local\Temp\ipykernel_69120\1783519745.py:46: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  tmpdf = (et_df.groupby([


New File


...

New File


C:\Users\49178\AppData\Local\Temp\ipykernel_69120\1783519745.py:46: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  tmpdf = (et_df.groupby([


New File


C:\Users\49178\AppData\Local\Temp\ipykernel_69120\1783519745.py:46: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  tmpdf = (et_df.groupby([


New File
One Iteration needs:  35.21875
4
Alles in:  168.53125


In [ ]:
#O_df.info()

In [ ]:
# then we add all the data frame is same condition in a big df

In [ ]:
files = glob.glob(f'{ONLY_EVENTS_DATA_PATH}\*.csv')

mylist = list()
for fi in files[:]: # reading all files
    mylist.append(fi)

mylist


['C:\\Users\\49178\\Downloads\\Telegram Desktop\\LoopARData\\FullLoopAR1\\HUDOnly\\HUDOnly_29006bddb3e8430582165c73bc7af864Autobahn.csv',
 'C:\\Users\\49178\\Downloads\\Telegram Desktop\\LoopARData\\FullLoopAR1\\HUDOnly\\HUDOnly_29006bddb3e8430582165c73bc7af864CountryRoad.csv',
 'C:\\Users\\49178\\Downloads\\Telegram Desktop\\LoopARData\\FullLoopAR1\\HUDOnly\\HUDOnly_29006bddb3e8430582165c73bc7af864MountainRoad.csv',
 'C:\\Users\\49178\\Downloads\\Telegram Desktop\\LoopARData\\FullLoopAR1\\HUDOnly\\HUDOnly_29006bddb3e8430582165c73bc7af864Westbrueck.csv',
 'C:\\Users\\49178\\Downloads\\Telegram Desktop\\LoopARData\\FullLoopAR1\\HUDOnly\\HUDOnly_4ed7e605b1614ccc98269519940f6965CountryRoad.csv',
 'C:\\Users\\49178\\Downloads\\Telegram Desktop\\LoopARData\\FullLoopAR1\\HUDOnly\\HUDOnly_4ed7e605b1614ccc98269519940f6965MountainRoad.csv',
 'C:\\Users\\49178\\Downloads\\Telegram Desktop\\LoopARData\\FullLoopAR1\\HUDOnly\\HUDOnly_4ed7e605b1614ccc98269519940f6965Westbrueck.csv',
 'C:\\Users\\491

In [ ]:
path =ONLY_EVENTS_DATA_PATH
csv_files = glob.glob(os.path.join(path, "*.csv"))
li = []

for f in sorted(files):
    df_al = pd.read_csv(f, index_col=None, header=0)
    li.append(df_al)
  #  df = df.reset_index(drop=True)
    #df_al['scene'] = os.path.basename(f)[43:52]
    #df_al['Condition'] = os.path.basename(f)[0:10]
    #df_al['Condition'] = Conditionname

    #df_al['Timestamp'] = pd.to_datetime(df['timestamp'])
    #df['Timestamp'] = df['timestamp'].dt.time
    #df['Timestamp']=  pd.to_datetime(df['Timestamp'],format="%H:%M:%S.%f")
    df_al = df_al.reset_index(drop=True)



df_al= pd.concat(li, axis=0, ignore_index=True)
ToDrop = ['time_from_start', 'TobiiTimeStamp', 'TobiiTimeStamp', 'eye_theta_h', 'eye_theta_v',
       'Equal',
       'RightEyeIsBlinkingWorld', 'RightEyeIsBlinkingLocal',
       'LeftEyeIsBlinkingWorld', 'LeftEyeIsBlinkingLocal', 'HmdPosition_x',
       'HmdPosition_y', 'HmdPosition_z', 'NoseVector_x', 'NoseVector_y',
       'NoseVector_z', 'EyePosWorldCombined_x', 'EyePosWorldCombined_y',
       'EyePosWorldCombined_z', 'EyeDirWorldCombined_x',
       'EyeDirWorldCombined_y', 'EyeDirWorldCombined_z',
       'EyePosLocalCombined_x', 'EyePosLocalCombined_y',
       'EyePosLocalCombined_z', 'EyeDirLocalCombined_x',
       'EyeDirLocalCombined_y', 'EyeDirLocalCombined_z', 'combinedEyeDir_x_intp', 'combinedEyeDir_y_intp',
       'combinedEyeDir_z_intp','EyeDirectionCombinedLocal_x_copy','EyeDirectionCombinedLocal_y_copy','EyeDirectionCombinedLocal_z_copy']

df_al = df_al.drop(ToDrop, axis=1)
df_al['timestamp'] = pd.to_datetime(df_al['timestamp'])

df_al['timestamp'] = pd.to_datetime(df_al.timestamp, utc=True, unit='s', origin='unix')
df_all= df_al.sort_values(by='timestamp', ascending=True, ignore_index=True)

df_all
#df_all[('timestamp')]= df_all[('timestamp')].values.astype("float64")
#df_all.timestamp.plot()



,timestamp,UnixTimeStamp,uid,SteeringInput,AcellerationInput,BrakeInput,eye_theta_h_int,eye_theta_v_int,Condition,Scene
0,2020-09-27 09:59:20.017987584+00:00,1.601201e+09,29006bddb3e8430582165c73bc7af864,-0.057362,0.0,0.0,0.225934,-0.017145,HUDOnly,MountainRoad
1,2020-09-27 09:59:23.021388032+00:00,1.601201e+09,29006bddb3e8430582165c73bc7af864,-0.023545,0.0,0.0,0.000000,0.000000,HUDOnly,MountainRoad
2,2020-09-27 09:59:23.031788288+00:00,1.601201e+09,29006bddb3e8430582165c73bc7af864,-0.023545,0.0,0.0,0.000000,0.000000,HUDOnly,MountainRoad
3,2020-09-27 09:59:23.053453312+00:00,1.601201e+09,29006bddb3e8430582165c73bc7af864,-0.023545,0.0,0.0,-0.056111,-0.094558,HUDOnly,MountainRoad
4,2020-09-27 09:59:23.075394560+00:00,1.601201e+09,29006bddb3e8430582165c73bc7af864,-0.023545,0.0,0.0,-0.054376,-0.074336,HUDOnly,MountainRoad
...,...,...,...,...,...,...,...,...,...,...
152860,2020-10-21 12:28:39.019028736+00:00,1.603283e+09,6f36a58eb7d843a593a01191c53bc0d0,-0.002252,0.0,0.0,-0.040326,-0.073123,HUDOnly,Autobahn
152861,2020-10-21 12:28:39.039972864+00:00,1.603283e+09,6f36a58eb7d843a593a01191c53bc0d0,-0.002252,0.0,0.0,-0.038731,-0.072490,HUDOnly,Autobahn
152862,2020-10-21 12:28:39.062911488+00:00,1.603283e+09,6f36a58eb7d843a593a01191c53bc0d0,-0.002711,0.0,0.0,-0.039282,-0.072217,HUDOnly,Autobahn
152863,2020-10-21 12:28:39.084851712+00:00,1.603283e+09,6f36a58eb7d843a593a01191c53bc0d0,-0.002711,0.0,0.0,-0.039130,-0.072645,HUDOnly,Autobahn


In [ ]:

df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152865 entries, 0 to 152864
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype              
---  ------             --------------   -----              
 0   timestamp          152865 non-null  datetime64[ns, UTC]
 1   UnixTimeStamp      152865 non-null  float64            
 2   uid                152865 non-null  object             
 3   SteeringInput      152865 non-null  float64            
 4   AcellerationInput  152865 non-null  float64            
 5   BrakeInput         152865 non-null  float64            
 6   eye_theta_h_int    152865 non-null  float64            
 7   eye_theta_v_int    152865 non-null  float64            
 8   Condition          152865 non-null  object             
 9   Scene              152865 non-null  object             
dtypes: datetime64[ns, UTC](1), float64(6), object(3)
memory usage: 11.7+ MB


In [ ]:
files = glob.glob(f'{SCENE_EVENT_DATA_PATH }\*.txt')
#ToDrop = [ 'StartofEventTimeStamp', 'EndOfEventTimeStamp']

Event_dur_df = pd.DataFrame()

#event_samples_df = pd.DataFrame()
for fi in files:
    with open(fi, 'r') as j:
        tmp = json.loads(j.read())
    for nS, sample in enumerate(tmp):
        x = tmp.get('EventBehavior')
    for p in range(len(x)):
            tmpdf = pd.json_normalize(data=x[p], sep='_')
           # interims_df = tmpdf[['EventName', 'StartofEventTimeStamp', 'EndOfEventTimeStamp',
          # 'EventDuration', 'SuccessfulCompletionState', 'HitObjectName']]
            tmpdf['uid'] = fi.split('\\')[-1].split('_')[0]
            tmpdf['scene'] = fi.split('_')[-1].split('.')[0]
            tmpdf['StartofEventTimeStamp'] = pd.to_datetime(tmpdf.StartofEventTimeStamp, utc=True, unit='s', origin='unix')
            #tmpdf['StartofEventTimeStamp'] = pd.to_datetime(tmpdf['StartofEventTimeStamp'])

            #tmpdf['Start'] =  tmpdf['Start'].dt.time


            tmpdf['EndOfEventTimeStamp'] = pd.to_datetime(tmpdf.EndOfEventTimeStamp, utc=True, unit='s', origin='unix')
            #tmpdf['Endoftimestamp'] = pd.to_datetime(tmpdf['Endoftimestamp'])

            #tmpdf['End'] =  tmpdf['End'].dt.time

           # tmpdf['EventName'] = 'Event_' + str(len(ns))
            Event_dur_df = pd.concat([Event_dur_df,tmpdf],ignore_index = True)


#cleaned_Event_df = Event_dur_df.drop(ToDrop, axis=1)
#Event_dur_df  = Event_dur_df.replace({'EventName' : {'Panne' : 'Event_1', 'GhostDriver' : 'Event_2', 'Baustelle' : 'Event_3'}})
Event_dur_df  = Event_dur_df.sort_values(by='StartofEventTimeStamp', ascending=True, ignore_index=True)
Event_dur_df.index = Event_dur_df.index + 1
#Event_dur_df

In [ ]:
i=1   # Efficiently iterating over rows in a Pandas DataFrame
for index, row in Event_dur_df.iterrows():
    Event_dur_df['EventName'][i]= 'Event_' + str(i)
    i+=1
Event_dur_df

C:\Users\49178\AppData\Local\Temp\ipykernel_144444\2346671845.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Event_dur_df['EventName'][i]= 'Event_' + str(i)


,EventName,StartofEventTimeStamp,EndOfEventTimeStamp,EventDuration,SuccessfulCompletionState,HitObjectName,uid,scene
1,Event_1,2020-09-27 09:59:39.846898432+00:00,2020-09-27 09:59:52.832177664+00:00,12.985279,True,,29006bddb3e8430582165c73bc7af864,MountainRoad
2,Event_2,2020-09-27 10:00:25.177758720+00:00,2020-09-27 10:00:37.190282752+00:00,12.012524,True,,29006bddb3e8430582165c73bc7af864,MountainRoad
3,Event_3,2020-09-27 10:01:30.801195776+00:00,2020-09-27 10:01:49.850981632+00:00,19.049786,True,,29006bddb3e8430582165c73bc7af864,MountainRoad
4,Event_4,2020-09-27 10:02:21.616264704+00:00,2020-09-27 10:02:24.541637888+00:00,2.925373,True,,29006bddb3e8430582165c73bc7af864,Westbrueck
5,Event_5,2020-09-27 10:02:50.679895552+00:00,2020-09-27 10:02:54.766573056+00:00,4.086677,False,Cube (32),29006bddb3e8430582165c73bc7af864,Westbrueck
6,Event_6,2020-09-27 10:03:24.108281344+00:00,2020-09-27 10:03:33.651193344+00:00,9.542912,True,,29006bddb3e8430582165c73bc7af864,Westbrueck
7,Event_7,2020-09-27 10:04:16.962380288+00:00,2020-09-27 10:04:33.387499776+00:00,16.425120,True,,29006bddb3e8430582165c73bc7af864,CountryRoad
8,Event_8,2020-09-27 10:05:09.237143040+00:00,2020-09-27 10:05:23.686411264+00:00,14.449268,True,,29006bddb3e8430582165c73bc7af864,CountryRoad
9,Event_9,2020-09-27 10:05:53.872447744+00:00,2020-09-27 10:05:58.918595072+00:00,5.046147,False,Motorcyclist2,29006bddb3e8430582165c73bc7af864,CountryRoad
10,Event_10,2020-09-27 10:07:16.569356800+00:00,2020-09-27 10:07:23.363255552+00:00,6.793899,True,,29006bddb3e8430582165c73bc7af864,Autobahn


In [ ]:
# The code processes event data, creates new DataFrames based on specific event timestamps, and saves these DataFrames to CSV files.

In [ ]:
Event_dur_df['StartofEventTimeStamp'] = pd.to_datetime(Event_dur_df.StartofEventTimeStamp).dt.tz_localize(None)
Event_dur_df['EndOfEventTimeStamp'] = pd.to_datetime(Event_dur_df.EndOfEventTimeStamp).dt.tz_localize(None)
df_all['timestamp'] = pd.to_datetime(df_all.timestamp).dt.tz_localize(None)
number = 1
eventmax = 58
df_list = list()

# start event loop
while number <= eventmax:

    # creat name for current event_x
    event_number = 'Event_' + str(number)
    # csv_name =  Conditionname +'_'+ i +'_'+ event_number + '_df.csv'
    csv_name = event_number + '_df.csv'



    print(event_number)

    # amount of appearances of event
    condi = Event_dur_df['EventName'] == event_number #gives true or false
    indexArray = Event_dur_df.index[condi] #gives array

    count = 0
    timestart_array = []
    timeend_array = []
    # create loop to get every stamp in array
    while count < len(indexArray):

        indexPart = indexArray[count]
        timestart_array.append(Event_dur_df.loc[indexPart, 'StartofEventTimeStamp'])
        timeend_array.append(Event_dur_df.loc[indexPart, 'EndOfEventTimeStamp'])

        count += 1

    upwards = 0
    lengta = len(timestart_array)
    mask_array = []
    mask_array2 = []

    while upwards < lengta:

        conStamp1 = df_all['timestamp'] >= timestart_array[upwards]
        conStamp2 = df_all['timestamp'] <= timeend_array[upwards]

        mask_array2 = df_all.index[conStamp1 & conStamp2]

        mask_array = np.append(mask_array, mask_array2, axis=None)

        upwards += 1

    # copy all relevant rows from EyeTracking towards new dataframe
    NewEvent1_df = df_all.iloc[mask_array,:]
    NewEvent1_df['Event'] = event_number
    display(NewEvent1_df)

    df_list.append(NewEvent1_df)

    NewEvent1_df.to_csv(Cut_EVENTS_DATA_PATH + "\\" + csv_name)

    number += 1

Event_1


C:\Users\49178\AppData\Local\Temp\ipykernel_144444\3167042610.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NewEvent1_df['Event'] = event_number


,timestamp,UnixTimeStamp,uid,SteeringInput,AcellerationInput,BrakeInput,eye_theta_h_int,eye_theta_v_int,Condition,Scene,Event
961,2020-09-27 09:59:39.851885824,1.601201e+09,29006bddb3e8430582165c73bc7af864,-0.063838,0.000000,0.0,0.351210,-0.119205,HUDOnly,MountainRoad,Event_1
962,2020-09-27 09:59:39.862855424,1.601201e+09,29006bddb3e8430582165c73bc7af864,-0.064297,0.000000,0.0,0.352694,-0.119988,HUDOnly,MountainRoad,Event_1
963,2020-09-27 09:59:39.879810816,1.601201e+09,29006bddb3e8430582165c73bc7af864,-0.064785,0.000000,0.0,0.353293,-0.119099,HUDOnly,MountainRoad,Event_1
964,2020-09-27 09:59:39.901752064,1.601201e+09,29006bddb3e8430582165c73bc7af864,-0.065702,0.000000,0.0,0.353007,-0.119299,HUDOnly,MountainRoad,Event_1
965,2020-09-27 09:59:39.911725568,1.601201e+09,29006bddb3e8430582165c73bc7af864,-0.066160,0.000000,0.0,0.356017,-0.120118,HUDOnly,MountainRoad,Event_1
...,...,...,...,...,...,...,...,...,...,...,...
1683,2020-09-27 09:59:52.739425536,1.601201e+09,29006bddb3e8430582165c73bc7af864,-0.029105,0.358480,0.0,0.002495,-0.080854,HUDOnly,MountainRoad,Event_1
1684,2020-09-27 09:59:52.762364416,1.601201e+09,29006bddb3e8430582165c73bc7af864,-0.030968,0.365353,0.0,0.002618,-0.080701,HUDOnly,MountainRoad,Event_1
1685,2020-09-27 09:59:52.772337920,1.601201e+09,29006bddb3e8430582165c73bc7af864,-0.032343,0.365353,0.0,0.002633,-0.080318,HUDOnly,MountainRoad,Event_1
1686,2020-09-27 09:59:52.795276288,1.601201e+09,29006bddb3e8430582165c73bc7af864,-0.034206,0.365353,0.0,0.002679,-0.080364,HUDOnly,MountainRoad,Event_1


Event_2


C:\Users\49178\AppData\Local\Temp\ipykernel_144444\3167042610.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NewEvent1_df['Event'] = event_number


...

C:\Users\49178\AppData\Local\Temp\ipykernel_144444\3167042610.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NewEvent1_df['Event'] = event_number


,timestamp,UnixTimeStamp,uid,SteeringInput,AcellerationInput,BrakeInput,eye_theta_h_int,eye_theta_v_int,Condition,Scene,Event
148501,2020-10-21 12:27:23.328756224,1.603283e+09,6f36a58eb7d843a593a01191c53bc0d0,-0.006865,0.0,0.0,0.086382,-0.053494,HUDOnly,Autobahn,Event_57
148502,2020-10-21 12:27:23.343715840,1.603283e+09,6f36a58eb7d843a593a01191c53bc0d0,-0.006865,0.0,0.0,0.086204,-0.051682,HUDOnly,Autobahn,Event_57
148503,2020-10-21 12:27:23.364660224,1.603283e+09,6f36a58eb7d843a593a01191c53bc0d0,-0.006865,0.0,0.0,0.085662,-0.050407,HUDOnly,Autobahn,Event_57
148504,2020-10-21 12:27:23.387598848,1.603283e+09,6f36a58eb7d843a593a01191c53bc0d0,-0.006865,0.0,0.0,0.087308,-0.051351,HUDOnly,Autobahn,Event_57
148505,2020-10-21 12:27:23.409540096,1.603283e+09,6f36a58eb7d843a593a01191c53bc0d0,-0.006865,0.0,0.0,0.087214,-0.051165,HUDOnly,Autobahn,Event_57
...,...,...,...,...,...,...,...,...,...,...,...
149158,2020-10-21 12:27:34.848912640,1.603283e+09,6f36a58eb7d843a593a01191c53bc0d0,0.008270,1.0,0.0,-0.057078,-0.121773,HUDOnly,Autobahn,Event_57
149159,2020-10-21 12:27:34.870855168,1.603283e+09,6f36a58eb7d843a593a01191c53bc0d0,0.007812,1.0,0.0,-0.056977,-0.118355,HUDOnly,Autobahn,Event_57
149160,2020-10-21 12:27:34.881825792,1.603283e+09,6f36a58eb7d843a593a01191c53bc0d0,0.007354,1.0,0.0,-0.055528,-0.113131,HUDOnly,Autobahn,Event_57
149161,2020-10-21 12:27:34.903755264,1.603283e+09,6f36a58eb7d843a593a01191c53bc0d0,0.006407,1.0,0.0,-0.055861,-0.114733,HUDOnly,Autobahn,Event_57


Event_58


C:\Users\49178\AppData\Local\Temp\ipykernel_144444\3167042610.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NewEvent1_df['Event'] = event_number


,timestamp,UnixTimeStamp,uid,SteeringInput,AcellerationInput,BrakeInput,eye_theta_h_int,eye_theta_v_int,Condition,Scene,Event
151880,2020-10-21 12:28:21.651170048,1.603283e+09,6f36a58eb7d843a593a01191c53bc0d0,0.000000,0.0,0.0,-0.039603,-0.082035,HUDOnly,Autobahn,Event_58
151881,2020-10-21 12:28:21.663137792,1.603283e+09,6f36a58eb7d843a593a01191c53bc0d0,0.000000,0.0,0.0,-0.037056,-0.075778,HUDOnly,Autobahn,Event_58
151882,2020-10-21 12:28:21.682087168,1.603283e+09,6f36a58eb7d843a593a01191c53bc0d0,0.000000,0.0,0.0,-0.033366,-0.069522,HUDOnly,Autobahn,Event_58
151883,2020-10-21 12:28:21.694055168,1.603283e+09,6f36a58eb7d843a593a01191c53bc0d0,0.000000,0.0,0.0,-0.036543,-0.072837,HUDOnly,Autobahn,Event_58
151884,2020-10-21 12:28:21.705026048,1.603283e+09,6f36a58eb7d843a593a01191c53bc0d0,0.000000,0.0,0.0,-0.036543,-0.072837,HUDOnly,Autobahn,Event_58
...,...,...,...,...,...,...,...,...,...,...,...
152707,2020-10-21 12:28:36.131775232,1.603283e+09,6f36a58eb7d843a593a01191c53bc0d0,0.019848,1.0,0.0,0.589538,-0.196347,HUDOnly,Autobahn,Event_58
152708,2020-10-21 12:28:36.141747712,1.603283e+09,6f36a58eb7d843a593a01191c53bc0d0,0.019848,1.0,0.0,0.561501,-0.207780,HUDOnly,Autobahn,Event_58
152709,2020-10-21 12:28:36.163689728,1.603283e+09,6f36a58eb7d843a593a01191c53bc0d0,0.018932,1.0,0.0,0.550855,-0.220317,HUDOnly,Autobahn,Event_58
152710,2020-10-21 12:28:36.186628864,1.603283e+09,6f36a58eb7d843a593a01191c53bc0d0,0.018474,1.0,0.0,0.529054,-0.198265,HUDOnly,Autobahn,Event_58


In [ ]:
files = glob.glob(f'{Cut_EVENTS_DATA_PATH }\*.csv')
li = []
for f in sorted(files):
    df_ev_Hud = pd.read_csv(f, index_col=[0] , header=0)   # to get rid of unnamed column we use index_col=[0]  index_col=['timestamp'] for indexes insteads of date

 #   df_ev_Hud['time_from_start'] = (df_ev_Hud.transform(lambda x: (x.index - x.index[0])/np.timedelta64(1,'s')) .UnixTimeStamp)
#    li.append(df_ev_Hud)
   # df_ev_Hud = df_ev_Hud.reset_index(drop=True)
#df_ev_Hud= pd.concat(li, axis=0, ignore_index=True)
df_ev_Hud  = df_ev_Hud.sort_values(by='timestamp', ascending=True, ignore_index=True)
df_ev_Hud['timestamp'] = pd.to_datetime(df_ev_Hud.timestamp).dt.tz_localize(None)
df_ev_Hud

,timestamp,UnixTimeStamp,uid,SteeringInput,AcellerationInput,BrakeInput,eye_theta_h_int,eye_theta_v_int,Condition,Scene,Event
0,2020-09-27 10:05:53.883418880,1.601201e+09,29006bddb3e8430582165c73bc7af864,0.000000,0.0,0.0,0.000000,0.000000,HUDOnly,CountryRoad,Event_9
1,2020-09-27 10:05:53.893391616,1.601201e+09,29006bddb3e8430582165c73bc7af864,0.000000,0.0,0.0,0.000000,0.000000,HUDOnly,CountryRoad,Event_9
2,2020-09-27 10:05:53.908352256,1.601201e+09,29006bddb3e8430582165c73bc7af864,0.000000,0.0,0.0,0.000000,0.000000,HUDOnly,CountryRoad,Event_9
3,2020-09-27 10:05:53.930293504,1.601201e+09,29006bddb3e8430582165c73bc7af864,0.000000,0.0,0.0,0.000000,0.000000,HUDOnly,CountryRoad,Event_9
4,2020-09-27 10:05:53.952234752,1.601201e+09,29006bddb3e8430582165c73bc7af864,0.000000,0.0,0.0,0.054365,-0.024499,HUDOnly,CountryRoad,Event_9
...,...,...,...,...,...,...,...,...,...,...,...
278,2020-09-27 10:05:58.817863936,1.601201e+09,29006bddb3e8430582165c73bc7af864,-0.062005,0.0,1.1,-0.086046,-0.103808,HUDOnly,CountryRoad,Event_9
279,2020-09-27 10:05:58.828834816,1.601201e+09,29006bddb3e8430582165c73bc7af864,-0.062922,0.0,1.1,-0.086046,-0.103808,HUDOnly,CountryRoad,Event_9
280,2020-09-27 10:05:58.850777088,1.601201e+09,29006bddb3e8430582165c73bc7af864,-0.066160,0.0,1.1,-0.090628,-0.104636,HUDOnly,CountryRoad,Event_9
281,2020-09-27 10:05:58.862744320,1.601201e+09,29006bddb3e8430582165c73bc7af864,-0.068024,0.0,1.1,-0.090628,-0.104636,HUDOnly,CountryRoad,Event_9


In [ ]:
#df_ev_Hud['UnixTimeStamp']=df_ev_Hud['UnixTimeStamp'].astype()
#df_ev_Hud['time_from_start'] = (df_ev_Hud.transform(lambda x: (x.index - x.index[0])/np.timedelta64(1,'s')) .UnixTimeStamp)
df_ev_Hud['UnixTimeStamp'] = pd.to_datetime(df_ev_Hud['UnixTimeStamp'], unit='s')
df_ev_Hud['time_from_start'] = (df_ev_Hud['UnixTimeStamp'] - df_ev_Hud['UnixTimeStamp'].min()) / np.timedelta64(1, 's')
df_ev_Hud

,timestamp,UnixTimeStamp,uid,SteeringInput,AcellerationInput,BrakeInput,eye_theta_h_int,eye_theta_v_int,Condition,Scene,Event,time_from_start
0,2020-09-27 10:05:53.883418880,2020-09-27 10:05:53.883418880,29006bddb3e8430582165c73bc7af864,0.000000,0.0,0.0,0.000000,0.000000,HUDOnly,CountryRoad,Event_9,0.000000
1,2020-09-27 10:05:53.893391616,2020-09-27 10:05:53.893391616,29006bddb3e8430582165c73bc7af864,0.000000,0.0,0.0,0.000000,0.000000,HUDOnly,CountryRoad,Event_9,0.009973
2,2020-09-27 10:05:53.908352256,2020-09-27 10:05:53.908352256,29006bddb3e8430582165c73bc7af864,0.000000,0.0,0.0,0.000000,0.000000,HUDOnly,CountryRoad,Event_9,0.024933
3,2020-09-27 10:05:53.930293504,2020-09-27 10:05:53.930293504,29006bddb3e8430582165c73bc7af864,0.000000,0.0,0.0,0.000000,0.000000,HUDOnly,CountryRoad,Event_9,0.046875
4,2020-09-27 10:05:53.952234752,2020-09-27 10:05:53.952234752,29006bddb3e8430582165c73bc7af864,0.000000,0.0,0.0,0.054365,-0.024499,HUDOnly,CountryRoad,Event_9,0.068816
...,...,...,...,...,...,...,...,...,...,...,...,...
278,2020-09-27 10:05:58.817863936,2020-09-27 10:05:58.817863936,29006bddb3e8430582165c73bc7af864,-0.062005,0.0,1.1,-0.086046,-0.103808,HUDOnly,CountryRoad,Event_9,4.934445
279,2020-09-27 10:05:58.828834816,2020-09-27 10:05:58.828834816,29006bddb3e8430582165c73bc7af864,-0.062922,0.0,1.1,-0.086046,-0.103808,HUDOnly,CountryRoad,Event_9,4.945416
280,2020-09-27 10:05:58.850777088,2020-09-27 10:05:58.850777088,29006bddb3e8430582165c73bc7af864,-0.066160,0.0,1.1,-0.090628,-0.104636,HUDOnly,CountryRoad,Event_9,4.967358
281,2020-09-27 10:05:58.862744320,2020-09-27 10:05:58.862744320,29006bddb3e8430582165c73bc7af864,-0.068024,0.0,1.1,-0.090628,-0.104636,HUDOnly,CountryRoad,Event_9,4.979325


In [ ]:
df_ev_Hud = df_ev_Hud.replace(['Event_13','Event_24','Event_36','Event_48' ],'Event_1')
df_ev_Hud = df_ev_Hud.replace(['Event_14','Event_25','Event_37','Event_49' ],'Event_2')
df_ev_Hud = df_ev_Hud.replace(['Event_15','Event_26','Event_38','Event_50' ],'Event_3')
df_ev_Hud = df_ev_Hud.replace(['Event_16','Event_27','Event_39','Event_51' ],'Event_4')
df_ev_Hud = df_ev_Hud.replace(['Event_17','Event_28','Event_40','Event_52' ],'Event_5')
df_ev_Hud = df_ev_Hud.replace(['Event_18','Event_29','Event_41','Event_53' ],'Event_6')
df_ev_Hud = df_ev_Hud.replace(['Event_19','Event_30','Event_42','Event_54' ],'Event_7')
df_ev_Hud = df_ev_Hud.replace(['Event_20','Event_31','Event_43','Event_55' ],'Event_8')
df_ev_Hud = df_ev_Hud.replace(['Event_21','Event_32','Event_44','Event_56' ],'Event_9')
df_ev_Hud = df_ev_Hud.replace(['Event_22','Event_33','Event_45','Event_57' ],'Event_10')
df_ev_Hud = df_ev_Hud.replace(['Event_23','Event_34','Event_46','Event_58' ],'Event_11')
df_ev_Hud = df_ev_Hud.replace(['Event_35','Event_47' ],'Event_12')

df_ev_Hud

,timestamp,uid,SteeringInput,AcellerationInput,BrakeInput,eye_theta_h_int,eye_theta_v_int,Condition,Scene,Event
0,2020-09-27 09:59:39.851885824,29006bddb3e8430582165c73bc7af864,-0.063838,0.0,0.0,0.351210,-0.119205,HUDOnly,MountainRoad,Event_1
1,2020-09-27 09:59:39.862855424,29006bddb3e8430582165c73bc7af864,-0.064297,0.0,0.0,0.352694,-0.119988,HUDOnly,MountainRoad,Event_1
2,2020-09-27 09:59:39.879810816,29006bddb3e8430582165c73bc7af864,-0.064785,0.0,0.0,0.353293,-0.119099,HUDOnly,MountainRoad,Event_1
3,2020-09-27 09:59:39.901752064,29006bddb3e8430582165c73bc7af864,-0.065702,0.0,0.0,0.353007,-0.119299,HUDOnly,MountainRoad,Event_1
4,2020-09-27 09:59:39.911725568,29006bddb3e8430582165c73bc7af864,-0.066160,0.0,0.0,0.356017,-0.120118,HUDOnly,MountainRoad,Event_1
...,...,...,...,...,...,...,...,...,...,...
33163,2020-10-21 12:28:36.131775232,6f36a58eb7d843a593a01191c53bc0d0,0.019848,1.0,0.0,0.589538,-0.196347,HUDOnly,Autobahn,Event_11
33164,2020-10-21 12:28:36.141747712,6f36a58eb7d843a593a01191c53bc0d0,0.019848,1.0,0.0,0.561501,-0.207780,HUDOnly,Autobahn,Event_11
33165,2020-10-21 12:28:36.163689728,6f36a58eb7d843a593a01191c53bc0d0,0.018932,1.0,0.0,0.550855,-0.220317,HUDOnly,Autobahn,Event_11
33166,2020-10-21 12:28:36.186628864,6f36a58eb7d843a593a01191c53bc0d0,0.018474,1.0,0.0,0.529054,-0.198265,HUDOnly,Autobahn,Event_11


In [ ]:
df_ev_Hud.to_csv('df_ev_Hud.csv')